# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
data_total = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName']).sum()
#data_total = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
data_total.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data_pivot = pd.pivot_table(data_total, values='Quantity', index='ProductName', columns='CustomerID', fill_value=0) 

In [6]:
data_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [7]:
#data_total['Apricots - Dried'][33]
#data_pivot.index
#squareform(pdist(data_total.T, 'euclidean'))
#data = data.set_index('CustomerID')

In [8]:
squareform(pdist(data_pivot.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [9]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

distances.head(3)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [10]:
similarities = distances[639].sort_values(ascending=False)[1:6]
similarities

CustomerID
3317    0.088530
3535    0.087779
3074    0.087047
3305    0.086333
649     0.085983
Name: 639, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [11]:
data_total = data_total.reset_index()
data_total.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [12]:
liste=[data_total.loc[data_total['CustomerID']==name] for name, score in dict(similarities).items()]
liste

[      CustomerID                      ProductName  Quantity
 2788        3317                         Bay Leaf         1
 2789        3317             Beef - Ground Medium         1
 2790        3317               Beef - Rib Eye Aaa         1
 2791        3317                Butter - Unsalted         1
 2792        3317      Cheese - Victor Et Berthold         1
 2793        3317                        Corn Meal         1
 2794        3317          Crab - Dungeness, Whole         1
 2795        3317          Crab - Imitation Flakes         1
 2796        3317               Crush - Cream Soda         1
 2797        3317     Cup - Translucent 7 Oz Clear         1
 2798        3317               Doilies - 5, Paper         1
 2799        3317            Ecolab - Solid Fusion         1
 2800        3317             Ezy Change Mophandle         1
 2801        3317                   Fenngreek Seed         1
 2802        3317             Garbage Bags - Clear         1
 2803        3317       

In [13]:
df = pd.DataFrame(liste)
data_select=pd.DataFrame()
for i in range(5):
    data_select=data_select.append(df[0][i])
#data_select

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [14]:
data_select = data_select.groupby('ProductName').agg({'Quantity' : sum}).reset_index().sort_values(by='Quantity',ascending=False)
data_select.head(10)

,ProductName,Quantity
49,Cod - Black Whole Fillet,4
208,Wine - Ej Gallo Sierra Valley,4
220,Yoghurt Tubes,3
131,"Pepper - Black, Whole",3
82,Halibut - Steaks,3
173,Soupfoamcont12oz 112con,3
212,"Wine - Magnotta, Merlot Sr Vqa",3
207,Wine - Crozes Hermitage E.,3
76,Garbage Bags - Clear,2
84,"Ice - Clear, 300 Lb For Carving",2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [15]:
data_pivot.reset_index()

CustomerID,ProductName,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,...,0,25,0,0,0,0,0,0,0,0
1,"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
2,Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,...,25,0,0,0,0,0,0,0,25,0
3,Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
4,Apricots - Dried,1,0,0,0,1,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
5,Apricots - Halves,0,0,1,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
6,Apricots Fresh,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,25
7,Arizona - Green Tea,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,25,0,0,0,0
8,Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Assorted Desserts,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,25,0,0,25


In [16]:
#distances[639] ==> le numero de client
#data_pivot
data_pivot_merge = data_pivot.merge(data_select, on='ProductName', how='inner')
data_pivot_merge[[639,'ProductName','Quantity']].loc[(data_pivot_merge[639]==0)&(data_pivot_merge['Quantity']!=0)].sort_values(by='Quantity',ascending=False)[:5]

,639,ProductName,Quantity
208,0,Wine - Ej Gallo Sierra Valley,4
49,0,Cod - Black Whole Fillet,4
220,0,Yoghurt Tubes,3
131,0,"Pepper - Black, Whole",3
207,0,Wine - Crozes Hermitage E.,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [17]:
reco_for_customer={}
def recommandation(customer): 
    similarities = distances[customer].sort_values(ascending=False)[1:6]

    liste=[data_total.loc[data_total['CustomerID']==name] for name, score in dict(similarities).items()]

    df = pd.DataFrame(liste)
    data_select=pd.DataFrame()
    
    for i in range(5):
        data_select=data_select.append(df[0][i])
    data_select_group = data_select.groupby('ProductName').agg({'Quantity' : sum}).reset_index().sort_values(by='Quantity',ascending=False)
    
    data_pivot_merge = data_pivot.merge(data_select_group, on='ProductName', how='inner')
    
    reco =data_pivot_merge[['ProductName','Quantity']].loc[(data_pivot_merge[customer]==0)&(data_pivot_merge['Quantity']!=0)].sort_values(by='Quantity',ascending=False)[:5]
    reco_dico=reco['ProductName'].to_dict()
    #je fais un "reset" des "keys" pour avoir partout [1,2,3,4,5]
    new_reco_dico=dict((i+1,reco_dico[k]) for i,k in enumerate(sorted(reco_dico.keys())))
    reco_for_customer[customer]=new_reco_dico
        
CustomerID_list=set(data.CustomerID)

for customer in CustomerID_list:
    recommandation(customer)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [18]:
#je crée une dataframe que je transpose pour faire de l'ID customer l'index
reco_to_df =pd.DataFrame.from_dict([reco_for_customer]).T
#ici je ne conserve que la valeur de chaque dico, je conserve les ID customer comme index et fais des "keys" le nom de colonne
reco_to_df = pd.DataFrame(reco_to_df[0].values.tolist(), index=reco_to_df.index)
reco_to_df.head(20)

,1,2,3,4,5
83973,Bandage - Fexible 1x3,Cocoa Butter,Ezy Change Mophandle,Pop Shoppe Cream Soda,Veal - Slab Bacon
59399,Cookie Chocolate Chip With,Knife Plastic - White,Nantucket - Pomegranate Pear,"Shrimp - Baby, Warm Water",Towels - Paper / Kraft
92168,Cup - Translucent 7 Oz Clear,Pop Shoppe Cream Soda,Snapple Lemon Tea,Spinach - Baby,"Veal - Brisket, Provimi,bnls"
49159,Cheese - Parmesan Grated,Cookies - Assorted,"Pasta - Penne, Rigate, Dry",Puree - Mocha,Rice - Jasmine Sented
18441,Beef - Texas Style Burger,"Garlic - Primerba, Paste","Lemonade - Natural, 591 Ml",Sausage - Breakfast,Soupfoamcont12oz 112con
22536,Bacardi Breezer - Tropical,Banana - Leaves,Cheese - Mozzarella,Chocolate - Dark,Table Cloth - 53x69 Colour
86028,Bread Foccacia Whole,"Orange - Canned, Mandarin",Pork - Hock And Feet Attached,Spinach - Baby,Vinegar - Tarragon
75791,Blueberries,Bread - Calabrese Baguette,Mussels - Cultivated,Turkey - Oven Roast Breast,Vaccum Bag 10x13
96272,Bread - English Muffin,Bread Crumbs - Panko,Ecolab - Lime - A - Way 4/4 L,Guinea Fowl,Sunflower Seed Raw
32785,Cookies - Assorted,Juice - Orange,Juice - V8 Splash,Table Cloth 54x72 White,Wine - Redchard Merritt


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [19]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'jaccard'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

reco_for_customer2={}
def recommandation(customer): 
    similarities = distances2[customer].sort_values(ascending=False)[1:6]

    liste=[data_total.loc[data_total['CustomerID']==name] for name, score in dict(similarities).items()]

    df = pd.DataFrame(liste)
    data_select=pd.DataFrame()
    
    for i in range(5):
        data_select=data_select.append(df[0][i])
    data_select_group = data_select.groupby('ProductName').agg({'Quantity' : sum}).reset_index().sort_values(by='Quantity',ascending=False)
    
    data_pivot_merge = data_pivot.merge(data_select_group, on='ProductName', how='inner')
    
    reco =data_pivot_merge[['ProductName','Quantity']].loc[(data_pivot_merge[customer]==0)&(data_pivot_merge['Quantity']!=0)].sort_values(by='Quantity',ascending=False)[:5]
    reco_dico=reco['ProductName'].to_dict()
    #je fais un "reset" des "keys" pour avoir partout [1,2,3,4,5]
    new_reco_dico=dict((i+1,reco_dico[k]) for i,k in enumerate(sorted(reco_dico.keys())))
    reco_for_customer2[customer]=new_reco_dico
        
CustomerID_list=set(data.CustomerID)

for customer in CustomerID_list:
    recommandation(customer)
    
#je crée une dataframe que je transpose pour faire de l'ID customer l'index
reco_to_df =pd.DataFrame.from_dict([reco_for_customer2]).T
#ici je ne conserve que la valeur de chaque dico, je conserve les ID customer comme index et fais des "keys" le nom de colonne
reco_to_df = pd.DataFrame(reco_to_df[0].values.tolist(), index=reco_to_df.index)
reco_to_df.head(20)

,1,2,3,4,5
83973,"Chocolate - Semi Sweet, Calets",Milk - 1%,Pork - Belly Fresh,Soup Knorr Chili With Beans,Towels - Paper / Kraft
59399,"Beans - Kidney, Canned","Cheese - Brie, Triple Creme",Cheese - Parmesan Cubes,"Oil - Shortening,liqud, Fry",Vanilla Beans
92168,Blackberries,Olives - Stuffed,"Potatoes - Instant, Mashed",Sprouts - Baby Pea Tendrils,Wiberg Super Cure
49159,Cheese - Bocconcini,Cookie Dough - Double,Flavouring - Orange,Macaroons - Two Bite Choc,Mussels - Cultivated
18441,Appetizer - Mushroom Tart,Bread - English Muffin,Bread Foccacia Whole,"Juice - Cranberry, 341 Ml",Nantucket - Pomegranate Pear
22536,Cocktail Napkin Blue,Isomalt,"Nut - Pistachio, Shelled",Pants Custom Dry Clean,Puree - Passion Fruit
86028,"Appetizer - Mini Egg Roll, Shrimp",Blueberries,Coffee - Dark Roast,Ice Cream Bar - Oreo Cone,Lettuce - Treviso
75791,Beef - Top Sirloin - Aaa,Beer - Labatt Blue,Phyllo Dough,Sea Bass - Whole,Tia Maria
96272,Bread - French Baquette,Knife Plastic - White,Langers - Ruby Red Grapfruit,Tea - Jasmin Green,Vinegar - Sherry
32785,Chips Potato Salt Vinegar 43g,Juice - Happy Planet,"Placemat - Scallop, White",Soup - Campbells Tomato Ravioli,Tomato - Tricolor Cherry
